# Supplementary: Comparing different gradient derivation methods

Related to Supplementary Figure S3

In [ ]:
import sys
sys.path.append("..")

from main import *

from numba import njit

if torch.cuda.is_available():  
    device = "cuda:0" 
else:  
    device = "cpu" 
    
print(device)

In [ ]:
from scipy.spatial.distance import pdist, squareform

def correlate_timeseries(A, B):
    A_mean = A.mean(axis=1, keepdims=True)
    B_mean = B.mean(axis=1, keepdims=True)
    A_std = A.std(axis=1, keepdims=True)
    B_std = B.std(axis=1, keepdims=True)
    A_norm = (A - A_mean) / A_std
    B_norm = (B - B_mean) / B_std
    return (A_norm @ B_norm.T) / A.shape[1]

def eta_squared_signals(a, b):
    m = (a + b) / 2
    M = np.mean(m)
    numerator = np.sum(((a - m) ** 2) + ((b - m) ** 2))
    denominator = np.sum(((a - M) ** 2) + ((b - M) ** 2))
    return 1 - (numerator / denominator)

def eta_squared(A):
    similarity = np.zeros((A.shape[0], A.shape[0]))
    for i in range(similarity.shape[0]):
        for j in range(similarity.shape[1]):
            similarity[i, j] = eta_squared_signals(A[i], A[j])
    return similarity

def cosine_similarity(x, normalized=False):
    cs = 1 - squareform(pdist(x, metric='cosine'))
    if normalized:
        ncs = 1 - np.arccos(cs, cs) / np.pi
        return ncs
    else:
        return cs

## Method 1: Changing the $\alpha$ parameter of the Diffusion Eigenmaps method

In [ ]:
vertices = np.load('../Files/vertices_ellipse.npy').astype('float')
eigenmodes = np.load('../Files/eigenmodes_ellipse.npy')

ellipse = Geometry(vertices, eigenmodes)

#### $\alpha=0$

In [ ]:
params = {'N_neurons': 1500,
          'h': 0.1,
          'g': 3,
          'tau': 3,
          'dale': False
          }

mode_similarity_matrices_alpha0 = []

for _ in range(10):

    simulator = Simulator(ellipse,
                          ChaoticRNN,
                          params)
    simulator.integrate(1000, output=False)
    C = simulator.compute_average_correlations(n_iters=100, T=500)
    mode_similarity, _ = simulator.compute_geometric_mapping(C, alpha=0)
    mode_similarity_matrices_alpha0.append(mode_similarity)

mode_similarity_matrices_alpha0 = np.stack(mode_similarity_matrices_alpha0, axis=0)

In [ ]:
plt.imshow(np.mean(np.abs(mode_similarity_matrices_alpha0), axis=0), cmap='Reds', vmin=0, vmax=1)

In [ ]:
np.save('../Results/supp_methods_similarity_alpha0.npy', np.mean(np.abs(mode_similarity_matrices_alpha0), axis=0))

#### $\alpha=0.5$

In [ ]:
params = {'N_neurons': 1500,
          'h': 0.1,
          'g': 3,
          'tau': 3,
          'dale': False
          }

mode_similarity_matrices_alpha05 = []

for _ in range(10):

    simulator = Simulator(ellipse,
                          ChaoticRNN,
                          params)
    simulator.integrate(1000, output=False)
    C = simulator.compute_average_correlations(n_iters=100, T=500)
    mode_similarity, _ = simulator.compute_geometric_mapping(C, alpha=0.5)
    mode_similarity_matrices_alpha05.append(mode_similarity)

mode_similarity_matrices_alpha05 = np.stack(mode_similarity_matrices_alpha05, axis=0)

In [ ]:
plt.imshow(np.mean(np.abs(mode_similarity_matrices_alpha05), axis=0), cmap='Reds', vmin=0, vmax=1)

In [ ]:
np.save('../Results/supp_methods_similarity_alpha05.npy', np.mean(np.abs(mode_similarity_matrices_alpha05), axis=0))

#### $\alpha=1.0$

In [ ]:
params = {'N_neurons': 1500,
          'h': 0.1,
          'g': 3,
          'tau': 3,
          'dale': False
          }

mode_similarity_matrices_alpha1 = []

for _ in range(10):

    simulator = Simulator(ellipse,
                          ChaoticRNN,
                          params)
    simulator.integrate(1000, output=False)
    C = simulator.compute_average_correlations(n_iters=100, T=500)
    mode_similarity, _ = simulator.compute_geometric_mapping(C, alpha=1)
    mode_similarity_matrices_alpha1.append(mode_similarity)

mode_similarity_matrices_alpha1 = np.stack(mode_similarity_matrices_alpha1, axis=0)

In [ ]:
plt.imshow(np.mean(np.abs(mode_similarity_matrices_alpha1), axis=0), cmap='Reds', vmin=0, vmax=1)

In [ ]:
np.save('../Results/supp_methods_similarity_alpha1.npy', np.mean(np.abs(mode_similarity_matrices_alpha1), axis=0))
np.save('../Results/supp_methods_corr_firingrates.npy', C)

## Method 2: Connectopic mapping of internal connectivity profiles

In [ ]:
params = {'N_neurons': 1500,
          'h': 0.1,
          'g': 3,
          'tau': 3,
          'dale': False
          }

mode_similarity_matrices_connectopic1 = []

for _ in range(10):

    simulator = Simulator(ellipse,
                          ChaoticRNN,
                          params)
    matrices = []
    for _ in tqdm(range(100)):
        simulator.reinitialize_dynamics(new_coordinates=False)
        simulator.integrate(500, output=False)
        timeseries = simulator.timeseries
        C = np.corrcoef(timeseries)
        C_sim = np.corrcoef(C)
        C_sim[np.diag_indices(C_sim.shape[0])] = 0
        matrices.append(C_sim)
    matrices = np.stack(matrices, axis=0)
    C = np.mean(np.abs(matrices), axis=0)
    mode_similarity_connectopic, _ = simulator.compute_geometric_mapping(C, alpha=0.5)
    mode_similarity_matrices_connectopic1.append(mode_similarity_connectopic)

mode_similarity_matrices_connectopic1 = np.stack(mode_similarity_matrices_connectopic1, axis=0)

In [ ]:
plt.imshow(np.mean(np.abs(mode_similarity_matrices_connectopic1), axis=0), cmap='Reds', vmin=0, vmax=1)

In [ ]:
np.save('../Results/supp_methods_similarity_connectopic1.npy', np.mean(np.abs(mode_similarity_matrices_connectopic1), axis=0))
np.save('../Results/supp_methods_corr_firingrates_connectopic.npy', np.corrcoef(timeseries))
np.save('../Results/supp_methods_profile_similarity.npy', C)

## Method 3: Connectopic mapping of "external" connectivity profiles

In [ ]:
class ExternalChaoticRNN:

    def __init__(self, coordinates, params, device='cuda:0'):
        self.device = device

        self.coordinates = coordinates
        N = self.coordinates.shape[0]
        self.tau = torch.tensor(params['tau']).float().to(device)
        self.g = torch.tensor(params['g']).float().to(device)
        self.h = params['h']
        self.dale = params['dale']

        W = generate_connectivity_matrix(coordinates, h=self.h, dale=self.dale)
        self.W = torch.tensor(W).float().to(device)

        self.x_0 = torch.from_numpy(np.random.uniform(-1, 1, (N, 1))).float().to(device)
        self.r_0 = self.phi(self.x_0)
        self.X = [self.x_0]
        self.R = [self.r_0]

    def integrate(self, T, input_timeseries, input_matrix, input_strength=3):
        for t in range(T):  # Euler integration scheme
            r = self.R[-1]
            x = self.X[-1]
            dx = (1 / self.tau) * (-x + self.g * torch.matmul(self.W, r) + input_strength * torch.matmul(input_matrix, input_timeseries[:, t:t+1]))
            self.X.append(x + dx)
            self.R.append(self.phi(self.X[-1]))
        return torch.concatenate(self.R, axis=1)

    def phi(self, x):
        return torch.tanh(x)


In [ ]:
params1 = {'N_neurons': 1500,
          'h': 0.1,
          'g': 3,
          'tau': 3,
          'dale': False
          }

params2 = {'N_neurons': 3000,
          'h': 0.1,
          'g': 0.5,
          'tau': 3,
          'dale': False
          }

ellipse1 = Geometry(vertices, eigenmodes)
ellipse2 = Geometry(vertices, eigenmodes)

In [ ]:
mode_similarity_matrices_connectopic2 = []

for _ in range(10):

    matrices = []
    
    simulator1 = Simulator(ellipse1,
                          ChaoticRNN,
                          params1)
    
    for _ in tqdm(range(100)):
        
        simulator1.reinitialize_dynamics(new_coordinates=False)
        simulator1.integrate(500, output=False)
        timeseries1 = simulator1.timeseries
        
        simulator2 = Simulator(ellipse2,
                          ExternalChaoticRNN,
                          params2)
        input_signals = torch.from_numpy(timeseries1).to(device).float()
        input_weights = torch.from_numpy(np.random.normal(0, 1 / np.sqrt(params1['N_neurons']),
                                                          (params2['N_neurons'], params1['N_neurons']))).to(device).float()
        timeseries2 = Simulator.torch_to_numpy(simulator2.dynamics.integrate(500, input_signals, input_weights))
        
        correlations = correlate_timeseries(timeseries1, timeseries2)
        C_sim = np.corrcoef(correlations)
        C_sim[np.diag_indices(C_sim.shape[0])] = 0
        matrices.append(C_sim)
    
    matrices = np.stack(matrices, axis=0)
    C = np.mean(np.abs(matrices), axis=0)
    mode_similarity_connectopic, _ = simulator1.compute_geometric_mapping(C, alpha=0.5)

    mode_similarity_matrices_connectopic2.append(mode_similarity_connectopic)

In [ ]:
plt.imshow(np.mean(np.abs(mode_similarity_matrices_connectopic2), axis=0), cmap='Reds', vmin=0, vmax=1)

In [ ]:
np.save('../Results/supp_methods_similarity_connectopic2.npy', np.mean(np.abs(mode_similarity_matrices_connectopic2), axis=0))
np.save('../Results/supp_methods_intersystem_correlations.npy', correlations)
np.save('../Results/supp_methods_profile_similarity2.npy', C)

# Loading data for figure

In [ ]:
mode_similarity_alpha0 = np.load('../Results/supp_methods_similarity_alpha0.npy')
mode_similarity_alpha05 = np.load('../Results/supp_methods_similarity_alpha05.npy')
mode_similarity_alpha1 = np.load('../Results/supp_methods_similarity_alpha1.npy')
C1 = np.load('../Results/supp_methods_corr_firingrates.npy')

mode_similarity_connectopic1 = np.load('../Results/supp_methods_similarity_connectopic1.npy')
C2 = np.load('../Results/supp_methods_corr_firingrates_connectopic.npy')
C_sim1 = np.load('../Results/supp_methods_profile_similarity.npy')

mode_similarity_connectopic2 = np.load('../Results/supp_methods_similarity_connectopic2.npy')
corr_intersystem = np.load('../Results/supp_methods_intersystem_correlations.npy')
C_sim2 = np.load('../Results/supp_methods_profile_similarity2.npy')

# Figure layout

In [ ]:
fig = PaperFigure(figsize=(7, 9), dpi=600)

fig.set_tick_length(1)
fig.set_font_size(6)
fig.add_background()

# Adding panels -------------------------------

w = 1
pad = (5 - 4 * w) / 3
fig.add_axes('corr1', (2 + 0 * (w + pad), 0), w, w)
fig.add_axes('modesim1', (2 + 1 * (w + pad), 0), w, w)
fig.add_axes('modesim2', (2 + 2 * (w + pad), 0), w, w)
fig.add_axes('modesim3', (2 + 3 * (w + pad), 0), w, w)

fig.add_axes('corr2', (2 + 0 * (w + pad), 1.5), w, w)
fig.add_axes('corrsim1', (2 + 1 * (w + pad), 1.5), w, w)
fig.add_axes('modesim4', (2 + 2 * (w + pad), 1.5), w, w)

fig.add_axes('corrinter', (2 + 0 * (w + pad), 3), 2 * w, w)
fig.add_axes('corrsim2', (2 + 2 * (w + pad), 3), w, w)
fig.add_axes('modesim5', (2 + 3 * (w + pad), 3), w, w)

fig.set_line_thickness(0.5)

# Content -------------------------------------

ax = fig.axes['corr1']
ax.imshow(C1, cmap='Reds', vmin=np.percentile(C, 5), vmax=np.percentile(C, 95))
ax.set_xticks([])
ax.set_yticks([])

print(np.percentile(C, 5), np.percentile(C, 95))

ax = fig.axes['modesim1']
ax.imshow(mode_similarity_alpha0, cmap='Reds', vmin=0, vmax=1)
ax.set_xticks([])
ax.set_yticks([])

ax = fig.axes['modesim2']
ax.imshow(mode_similarity_alpha05, cmap='Reds', vmin=0, vmax=1)
ax.set_xticks([])
ax.set_yticks([])

ax = fig.axes['modesim3']
ax.imshow(mode_similarity_alpha1, cmap='Reds', vmin=0, vmax=1)
ax.set_xticks([])
ax.set_yticks([])

ax = fig.axes['corr2']
ax.imshow(C1, cmap='Reds', vmin=np.percentile(C1, 5), vmax=np.percentile(C1, 95))
ax.set_xticks([])
ax.set_yticks([])

ax = fig.axes['corrsim1']
ax.imshow(C_sim1, cmap='Reds', vmin=np.percentile(C_sim1, 10), vmax=np.percentile(C_sim1, 95))
ax.set_xticks([])
ax.set_yticks([])

print(np.percentile(C_sim1, 10), np.percentile(C_sim1, 95))

ax = fig.axes['modesim4']
ax.imshow(mode_similarity_connectopic1, cmap='Reds', vmin=0, vmax=1)
ax.set_xticks([])
ax.set_yticks([])

ax = fig.axes['corrinter']
ax.imshow(corr_intersystem, vmin=-0.1, vmax=0.1, cmap='coolwarm')
ax.set_xticks([])
ax.set_yticks([])

ax = fig.axes['corrsim2']
ax.imshow(C_sim2, cmap='Reds', vmin=np.percentile(C_sim2, 10), vmax=np.percentile(C_sim2, 95))
ax.set_xticks([])
ax.set_yticks([])

print(np.percentile(C_sim2, 10), np.percentile(C_sim2, 95))

ax = fig.axes['modesim5']
ax.imshow(mode_similarity_connectopic2, cmap='Reds', vmin=0, vmax=1)
ax.set_xticks([])
ax.set_yticks([])

# Displaying ----------------------------------

fig.save('../Figures/supp_gradient_methods_incomplete.svg')

fig.show()